In [1]:
from sklearn import preprocessing
from sklearn import cross_validation
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction.text import CountVectorizer

In [5]:
########## STEP 1: DATA IMPORT AND PREPROCESSING ##########

# Here we're taking in the training data and splitting it into two lists: One with the text of
# each bill title, and the second with each bill title's corresponding category. Order is important.
# The first bill in list 1 should also be the first category in list 2.
training = [line.strip().split('|') for line in open('bills_training.txt', 'r').readlines()]
text = [t[0] for t in training if len(t) > 1]
labels = [t[1] for t in training if len(t) > 1]

# A little bit of cleanup for scikit-learn's benefit. Scikit-learn models wants our categories to
# be numbers, not strings. The LabelEncoder performs this transformation.
encoder = preprocessing.LabelEncoder()
correct_labels = encoder.fit_transform(labels)

In [6]:
########## STEP 2: FEATURE EXTRACTION ##########
vectorizer = CountVectorizer(stop_words='english')
data = vectorizer.fit_transform(text)


In [7]:
########## STEP 3: MODEL BUILDING ##########
model = DecisionTreeClassifier()
fit_model = model.fit(data, correct_labels)


In [8]:
# ########## STEP 4: EVALUATION ##########
# Evaluate our model with 10-fold cross-validation
scores = cross_validation.cross_val_score(model, data, correct_labels, cv=5)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))


/Users/olayaargueso/.virtualenvs/algos/lib/python3.5/site-packages/sklearn/cross_validation.py:516: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of labels for any class cannot be less than n_folds=5.
  % (min_labels, self.n_folds)), Warning)


Accuracy: 0.65 (+/- 0.04)


In [9]:
# ########## STEP 5: APPLYING THE MODEL ##########
docs_new = ["Public postsecondary education: executive officer compensation.",
            "An act to add Section 236.3 to the Education code, related to the pricing of college textbooks.",
            "Political Reform Act of 1974: campaign disclosures.",
            "An act to add Section 236.3 to the Penal Code, relating to human trafficking."
        ]

test_data = vectorizer.transform(docs_new)

for i in range(len(docs_new)):
    print('%s -> %s' % (docs_new[i], encoder.classes_[model.predict(test_data.toarray()[i])]))
   

Public postsecondary education: executive officer compensation. -> ['Education']
An act to add Section 236.3 to the Education code, related to the pricing of college textbooks. -> ['Education']
Political Reform Act of 1974: campaign disclosures. -> ['Campaign Finance and Election Issues']
An act to add Section 236.3 to the Penal Code, relating to human trafficking. -> ['Crime']


/Users/olayaargueso/.virtualenvs/algos/lib/python3.5/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/Users/olayaargueso/.virtualenvs/algos/lib/python3.5/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/Users/olayaargueso/.virtualenvs/algos/lib/python3.5/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature 

I can understand how useful something like this could be if working with really huge datasets, since it seems helpful to automatise basic tasks (like labelling bills or tagging emails). However, I wonder whether this would work for any language other than English (my bet is it would not). 